In [15]:
# 1. Пілготовка датасету

import os
import shutil
from pathlib import Path

# Шлях до оригінального набору даних
original_dataset_dir = 'dogs-vs-cats\\train\\train'

# Шляхи до директорій для підготовки скороченого набору даних
base_dir = 'dogs-vs-cats-dataset'
os.makedirs(base_dir, exist_ok=True)

# Створюємо структуру папок для тренування, валідації та тестування
train_dir = os.path.join(base_dir, 'train')
os.makedirs(train_dir, exist_ok=True)
validation_dir = os.path.join(base_dir, 'validation')
os.makedirs(validation_dir, exist_ok=True)
test_dir = os.path.join(base_dir, 'test')
os.makedirs(test_dir, exist_ok=True)

# Папки для кішок і собак у тренуванні, валідації та тестуванні
train_cats_dir = os.path.join(train_dir, 'cats')
os.makedirs(train_cats_dir, exist_ok=True)
train_dogs_dir = os.path.join(train_dir, 'dogs')
os.makedirs(train_dogs_dir, exist_ok=True)

validation_cats_dir = os.path.join(validation_dir, 'cats')
os.makedirs(validation_cats_dir, exist_ok=True)
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
os.makedirs(validation_dogs_dir, exist_ok=True)

test_cats_dir = os.path.join(test_dir, 'cats')
os.makedirs(test_cats_dir, exist_ok=True)
test_dogs_dir = os.path.join(test_dir, 'dogs')
os.makedirs(test_dogs_dir, exist_ok=True)

# Копіюємо перші 2000 зображень кішок для тренування
fnames = ['cat.{}.jpg'.format(i) for i in range(2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_cats_dir, fname)
    shutil.copyfile(src, dst)

# Копіюємо перші 2000 зображень собак для тренування
fnames = ['dog.{}.jpg'.format(i) for i in range(2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_dogs_dir, fname)
    shutil.copyfile(src, dst)

# Копіюємо перші 500 зображень для валідації кішок
fnames = ['cat.{}.jpg'.format(i) for i in range(2000, 2500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_cats_dir, fname)
    shutil.copyfile(src, dst)

# Копіюємо перші 500 зображень для валідації собак
fnames = ['dog.{}.jpg'.format(i) for i in range(2000, 2500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_dogs_dir, fname)
    shutil.copyfile(src, dst)

# Копіюємо перші 500 зображень для тестування кішок
fnames = ['cat.{}.jpg'.format(i) for i in range(2500, 3000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_cats_dir, fname)
    shutil.copyfile(src, dst)

# Копіюємо перші 500 зображень для тестування собак
fnames = ['dog.{}.jpg'.format(i) for i in range(2500, 3000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_dogs_dir, fname)
    shutil.copyfile(src, dst)

In [17]:
# 2. Підготовка генераторів для тренування та валідації

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Генератор для тренувальних зображень з аугментацією
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Генератор для валідації без аугментації
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

Found 4000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [19]:
# 3. Перенесення навчання з VGG16

from tensorflow.keras.applications import VGG16
from tensorflow.keras import models, layers, optimizers

# Завантажуємо модель VGG16 без верхніх шарів (головної частини)
conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Додаємо нові шари для класифікації
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

# Заморожуємо шари базової моделі (VGG16)
conv_base.trainable = False

# Компіляція моделі
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=2e-5),
              metrics=['accuracy'])

# Навчання моделі
history = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=50
)

Epoch 1/30


D:\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


100/100 ━━━━━━━━━━━━━━━━━━━━ 168s 2s/step - accuracy: 0.5768 - loss: 0.7112 - val_accuracy: 0.7960 - val_loss: 0.4920
Epoch 2/30


D:\anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


100/100 ━━━━━━━━━━━━━━━━━━━━ 71s 704ms/step - accuracy: 0.6978 - loss: 0.5889 - val_accuracy: 0.7970 - val_loss: 0.4679
Epoch 3/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 164s 2s/step - accuracy: 0.7172 - loss: 0.5460 - val_accuracy: 0.8380 - val_loss: 0.3924
Epoch 4/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 68s 678ms/step - accuracy: 0.7485 - loss: 0.5033 - val_accuracy: 0.8450 - val_loss: 0.3779
Epoch 5/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 163s 2s/step - accuracy: 0.7408 - loss: 0.5165 - val_accuracy: 0.8500 - val_loss: 0.3497
Epoch 6/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 68s 678ms/step - accuracy: 0.7925 - loss: 0.4473 - val_accuracy: 0.8570 - val_loss: 0.3457
Epoch 7/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 163s 2s/step - accuracy: 0.7824 - loss: 0.4515 - val_accuracy: 0.8450 - val_loss: 0.3417
Epoch 8/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 68s 678ms/step - accuracy: 0.7908 - loss: 0.4429 - val_accuracy: 0.8630 - val_loss: 0.3246
Epoch 9/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 163s 2s/step - accuracy: 0.8162 - loss: 0.4049 - val_accurac

In [21]:
# 4. Fine Tuning

# Розморозимо останні шари для Fine-Tuning
conv_base.trainable = True

for layer in conv_base.layers[:15]:
    layer.trainable = False

# Компіляція моделі після Fine-Tuning
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=1e-5),
              metrics=['accuracy'])

# Донавчання моделі
history_fine = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=50
)

Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 191s 2s/step - accuracy: 0.8468 - loss: 0.3429 - val_accuracy: 0.9080 - val_loss: 0.2169
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 73s 723ms/step - accuracy: 0.8777 - loss: 0.2749 - val_accuracy: 0.9070 - val_loss: 0.2381
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 184s 2s/step - accuracy: 0.8748 - loss: 0.2988 - val_accuracy: 0.9100 - val_loss: 0.2073
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 73s 724ms/step - accuracy: 0.8603 - loss: 0.2796 - val_accuracy: 0.9290 - val_loss: 0.1932
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 184s 2s/step - accuracy: 0.8980 - loss: 0.2404 - val_accuracy: 0.9270 - val_loss: 0.1854
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 73s 727ms/step - accuracy: 0.8951 - loss: 0.2473 - val_accuracy: 0.9170 - val_loss: 0.2299
Epoch 7/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 184s 2s/step - accuracy: 0.8923 - loss: 0.2487 - val_accuracy: 0.9350 - val_loss: 0.1815
Epoch 8/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 73s 726ms/step - accuracy: 0.9040 - loss: 0.2405 - 

In [23]:
# Розпізнавання зображень

# Генератор для тестових зображень (без аугментації)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

# Оцінка моделі на тестовому наборі даних
test_loss, test_acc = model.evaluate(test_generator, steps=50)
print(f"Test accuracy: {test_acc:.4f}")

Found 1000 images belonging to 2 classes.
50/50 ━━━━━━━━━━━━━━━━━━━━ 37s 736ms/step - accuracy: 0.9018 - loss: 0.2471
Test accuracy: 0.9080


In [25]:
# Збереження моделі

# Зберігаємо модель на диск
model.save('cats_and_dogs_model.h5')